In [1]:
import json
import os


In [6]:
answer_mapping = {
    "A": 0,
    "B": 1,
    "C": 2,
    "D": 3
}

class Sample:
    def __init__(self, sample_id, answer, options, article, generated_utterances=None):
        self.sample_id = sample_id
        self.answer = answer
        self.options = options
        self.article = article
        
        self.label_descr = options[answer_mapping[answer]]
        
        self.generated_utterances = generated_utterances or []

    def __str__(self):
        if self.generated_utterances:
            return f"Sample ID: {self.sample_id}\nAnswer Number: {self.answer}\nAnswer: {self.label_descr}\nOptions: {self.options}\nContext: {self.article}\nGenerated Utterances: {self.generated_utterances}"
        else:
            return f"Sample ID: {self.sample_id}\nAnswer Number: {self.answer}\nAnswer: {self.label_descr}\nOptions: {self.options}\nContext: {self.article}"
            
def load_sample_from_file(filename):
    with open(filename, 'r') as file:
        data = json.loads(file.read())
    return Sample(
        data['id'],
        data['answers'],
        data['options'],
        data['article']
    )

def load_all_samples(base_dir,mode):
    data_dir = os.path.join(base_dir, mode)
    samples = []
    for filename in os.listdir(data_dir):
        if filename.endswith('.txt'):
            sample = load_sample_from_file(os.path.join(data_dir, filename))
            samples.append(sample)
    return samples


In [7]:
def compute_averages(samples):
    total_word_count = 0
    total_sentences = 0
    total_turns = 0

    for sample in samples:
        # average sentence length for the responses
        for option in sample.options:
            total_word_count += len(option.split())
            total_sentences += 1

        # average turns in a conversation
        turns = sample.article.split('.')
        total_turns += len([t for t in turns if t.strip()])  # Filtering out any empty turns

    avg_sentence_length = total_word_count / total_sentences
    avg_turns = total_turns / len(samples)

    return avg_sentence_length, avg_turns

In [8]:
base_dir = r"data/mutual"
train_samples = load_all_samples(base_dir, "train")

for i in range(2):
    print(train_samples[i])
    print("   ")

Sample ID: train_5844
Answer Number: A
Answer: m : alright . have a nice time shopping with your mother then .
Options: ['m : alright . have a nice time shopping with your mother then .', "m : let 's have breakfast first and then ride along the lake . see you in a bit !", "m : i did n't know you went shopping with your mother yesterday .", 'm : have a nice time celebrating the new year with your mother this morning .']
Context: m : would you like to go to right along the lake with me this morning ? f : i 'd like to but i have to go to the city mall with my mother . the new year is coming .
   
Sample ID: train_4582
Answer Number: A
Answer: f : it must be something wrong with the engine so you could n't get your car going .
Options: ["f : it must be something wrong with the engine so you could n't get your car going .", 'f : how much did it cost you to repair your car ?', "f : i 'm sorry to hear that you had an accident this morning , is everything all right ?", "f : since you could n't

In [9]:
avg_sentence_length, avg_turns = compute_averages(train_samples)

print(f"Average sentence length of responses: {avg_sentence_length:.2f} words")
print(f"Average turns per conversation: {avg_turns:.2f}")

Average sentence length of responses: 17.87 words
Average turns per conversation: 6.66


In [10]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch 
#from .autonotebook import tqdm as notebook_tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
model = GPT2LMHeadModel.from_pretrained("gpt2-medium")

next_utterance_approx_len = 30

model.to(device)
model.eval() 

def generate_utterances_for_sample(sample, num_sequences=3):
   
    input_text = sample.article
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device) 

    output = model.generate(input_ids, max_length=len(input_ids[0]) + next_utterance_approx_len, num_return_sequences=num_sequences, pad_token_id=tokenizer.eos_token_id,
                           num_beams=3, do_sample=True,top_k=50, temperature=1.5)
    
    generated_utterances =  [tokenizer.decode(o[len(input_ids[0]):]) for o in output]
    return generated_utterances

In [25]:
def augment_samples(sample):
    #for sample in samples:
        utterances = generate_utterances_for_sample(sample)
        for ut in utterances:
            print(ut)
            print("-" * 50)
        sample.generated_utterances = utterances
        print(sample.generated_utterances)

augment_samples(train_samples[2])

#for sample in samples[:1]:
    #print(sample)
    #print("-" * 50)

 it's quite hot out in the sun. i'm gonna have to use the restroom. m : what's your name?..
--------------------------------------------------
 i want to get off at the airport but i can't because i haven't got a ticket. m : i know i should get you a ticket
--------------------------------------------------
 i'm not wearing any sunglasses. i'm sorry. m : i know you have no idea what you're talking about. but why don '
--------------------------------------------------
[" it's quite hot out in the sun. i'm gonna have to use the restroom. m : what's your name?..", " i want to get off at the airport but i can't because i haven't got a ticket. m : i know i should get you a ticket", " i'm not wearing any sunglasses. i'm sorry. m : i know you have no idea what you're talking about. but why don '"]


In [30]:
train_samples[2].article

"f : why have you got your sunglasses on ? it 's not that sunny outside . m : i know it 's quite cool today . but i 've got a terrible headache and my eyes hurt in the light ."

In [31]:
import time
import openai

openai.api_key = "your key"
gpt_version = "gpt-3.5-turbo"
text = train_samples[2].article

def prompt_llm(prompt, max_tokens = 64, temperature=0, stop=None):
  response = openai.Completion.create(engine=gpt_version, prompt=prompt,   max_tokens=max_tokens, temperature=temperature, stop=stop)
  time.sleep(1)
  return response["choices"][0]["text"].strip()

prompt_llm(text, max_tokens = 30, temperature = 0)

RateLimitError: You exceeded your current quota, please check your plan and billing details.